Noise model notebook for the SuperCDMS ionization readout.

![title](img/charge_circuit.png)

In [ ]:
%matplotlib notebook

#!/bin/python

import sys
import argparse
import cmath
import numpy as np
import matplotlib.pyplot as plt
from lib import impedance as Q


In [ ]:
logy = True
T_4K = 4.0
T_300K = 300.0
drainI = 1e-3

First create some helper functions

In [ ]:
def to_np_array(func, arr):
    return np.array([func(a) for a in arr])

In [ ]:
def impedancePlot(fig, f, Z, name=None, ylabel='Impedance', xlabel='Frequency (Hz)', note=None, legend=None):
    """Plot impedance and phase"""


    if fig == None:
        plt.figure()
    
    ax_MC211 = plt.subplot(211)
    line1, = plt.plot(f, np.abs(Z), label=legend)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    if logy:
        ax_MC211.set_yscale(u'log')
    ax_MC211.set_xscale(u'log')
    if note:
        plt.text(0.1, 0.9, note, transform = ax_MC211.transAxes)
    if legend:
        #hs = ax_MC211.get_legend_handles_labels()
        #hs = [hs,line1]
        #if hss:
        #    handles.append(line1)
        plt.legend()
        #else:
        #    plt.legend(handles=[line1])
    

    Z_phase = to_np_array(cmath.phase, Z)

    ax_MC212 = plt.subplot(212)
    plt.plot(f, Q.radToDeg(Z_phase))
    plt.ylabel('Phase')
    plt.xlabel('Frequency')
    #ax_MC1.set_yscale(u'log')
    ax_MC212.set_xscale(u'log')

    if name:
        plt.savefig(name,bbox_inches='tight')

    
    return [ax_MC211, ax_MC212]

In [ ]:
def noisePlot(fig, f, Z, name=None, ylabel='Noise', xlabel='Frequency', note=None, legend=None):
    """Plot noise."""

    if fig == None:
        plt.figure()

    
    ax_MC211 = plt.subplot(111)
    line1, = plt.plot(f, np.abs(Z), label=legend)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    if logy:
        ax_MC211.set_yscale(u'log')
    ax_MC211.set_xscale(u'log')
    if note:
        plt.text(0.1, 0.9, note, transform = ax_MC211.transAxes)
    if legend:
        #hs = ax_MC211.get_legend_handles_labels()
        #hs = [hs,line1]
        #if hss:
        #    handles.append(line1)
        plt.legend()
        #else:
        #    plt.legend(handles=[line1])

    if name:
        plt.savefig(name,bbox_inches='tight')


    return [ax_MC211]
              




Initialize all the circuits that we will use.

In [ ]:
Z1_MC = Q.Z1_MC('Z1_MC')
Z1_g = Q.Z1_g('Z1_g')
Z2 = Q.Z2('Z2')
Z3 = Q.Z3('Z3')
Z4 = Q.Z4('Z4')
Z5 = Q.Z5('Z5')
Z6 = Q.Z6('Z6')
opamp = Q.LT1677('opamp')
bjt = Q.BJT('bjt')
hemt = Q.HEMT('HEMT')

Create the frequency array we use for most plots vs freq.

In [ ]:
f_arr = np.logspace(0,6)
#f_arr = np.append(np.logspace(0,4)[:-1],np.logspace(4,5))

Calculate opamp gain

In [ ]:
Aopen = to_np_array(opamp.Aopen, f_arr)
impedancePlot(plt.figure(), f_arr, Aopen, 'Aopen_opamp.png', ylabel='Aopen opamp', note='Aopen opamp')


Feedback setup for the opamp below.

In [ ]:

Z_Z5 = to_np_array(Z5.Z_tot,f_arr)
Z_Z6 = to_np_array(Z6.Z_tot,f_arr)
fig = plt.figure()
impedancePlot(fig, f_arr, Z_Z5, legend=r'$Z_{5}$')
impedancePlot(fig, f_arr, Z_Z6, 'Z_Z5_Z6.png', legend=r'$Z_{6}$')

Feedback fraction for the opamp
There is no damping because the opamp is non-inverting.

In [ ]:
B = Z_Z5/(Z_Z5 +Z_Z6)
fig = plt.figure()
impedancePlot(fig, f_arr, B, 'B.png',ylabel='Opamp feedback fraction B',note='Feedback fraction B')
#fig.clf()



opamp closed loop gain

In [ ]:
Aclosed = Aopen/(1 + Aopen*B)

impedancePlot(plt.figure(), f_arr, Aclosed, 'Aclosed_opamp.png', ylabel='Aclosed opamp', note='Aclosed opamp')



## Voltage gain of HEMT. 
Gain given by gm times load

In [ ]:
# load impedance
Z_Z3 = to_np_array(Z3.Z_tot, f_arr)

# take into account any load impedance from current mirror in series 
Z_BJT = to_np_array(bjt.Z_tot, f_arr)
Z_load = Z_Z3 + Z_BJT

# any impedance from mirror would split voltage gain at opamp input
Aopen_HEMT = hemt.gm*Z_load*(Z_Z3/Z_load)

fig=plt.figure()
impedancePlot(fig, f_arr, Z_Z3, 'Z3.png', note='Z3')
fig=plt.figure()
impedancePlot(fig, f_arr, Z_load, 'Z_load.png', note='Z_load')
fig=plt.figure()
impedancePlot(fig, f_arr, Aopen_HEMT, 'Aopen_HEMT.png', ylabel='Aopen HEMT', note='Aopen HEMT gm={0:.1f}mS'.format(hemt.gm*1e3))


## Total voltage gain

Total open loop gain is product of closed opamp gain and open HEMT gain

In [ ]:
Atotal_open = Aclosed*Aopen_HEMT

impedancePlot(plt.figure(), f_arr, Atotal_open, 'Atotal_open.png', ylabel='Gain', note='Total open loop gain')






### Feedback for inverting amplifier

HEMT input impedance: input capacitance and input resistor are parallel to ground

In [ ]:
Z_HEMT = to_np_array(hemt.Z_tot, f_arr) 
impedancePlot(plt.figure(), f_arr, Z_HEMT, 'Z_HEMT.png', note=r'$Z_{HEMT}$')

Impedance of gate coupling capacitor to the HEMT gate input

In [ ]:
Z_Z1_g = to_np_array(Z1_g.Z_tot, f_arr) 
impedancePlot(plt.figure(), f_arr, Z_Z1_g, 'Z1_g.png', note=r'$Z_{1g}$')

The gate coupling capacitor is in series with an open connection

In [ ]:
Ropen = Q.Resistor(1e19,'Ropen')
Z_Z1_open = Q.series(Z_Z1_g, Ropen.Z(f_arr))

impedancePlot(plt.figure(), f_arr, Z_Z1_open, 'Z1_open.png', note=r'$Z_{1,open}$')

The feedback signal are split between the HEMT input impedance and gate coupling capactor

In [ ]:
Z_input_4K = Q.parallel(Z_HEMT, Z_Z1_open)
impedancePlot(plt.figure(), f_arr, Z_input_4K, 'Z_input_4K.png', note=r'$Z_{input,4K}$')


Feedback impedance

In [ ]:
Z_Z2 = to_np_array(Z2.Z_tot, f_arr) 

impedancePlot(plt.figure(), f_arr, Z_Z2, 'Z2.png', note=r'$Z_{2}$')


Feedback fraction and damping

In [ ]:
H_fb = Z_input_4K/(Z_input_4K + Z_Z2)
H_in = -1*Z_Z2/(Z_input_4K + Z_Z2)

fig = plt.figure()
impedancePlot(fig, f_arr, H_fb, legend=r'$H_{fb}$')
impedancePlot(fig, f_arr, H_in, 'H_in_fb.png', legend=r'$H_{in}$', ylabel='Feedback fraction & damping')



### total closed loop voltage gain

In [ ]:
Atotal_closed = H_in*Atotal_open/(1+Atotal_open*H_fb)

impedancePlot(fig, f_arr, Atotal_closed, 'Atotal_closed.png', ylabel='Gain', note='Total closed')
fig.clf()


Cross check effect from damping and feedback fraction

In [ ]:
Atotal_closed_no_H_in = Atotal_open/(1+Atotal_open*H_fb)
Atotal_closed_no_H_fb = H_in*Atotal_open/(1+Atotal_open)

fig=plt.figure()
impedancePlot(fig, f_arr, Atotal_closed,legend='Total closed')
impedancePlot(fig, f_arr, Atotal_closed_no_H_in, legend=r'No $H_{in}$')
impedancePlot(fig, f_arr, Atotal_closed_no_H_fb, ylabel='Gain', legend=r'No $H_{fb}$',note='Compare FB/damping effect.', name='Atotal_closed_cmp.png')


### Include circuitry at MC stage

Calculate impedance contribution from  circuitry at the MC stage

In [ ]:
Z_Z1_MC = to_np_array(Z1_MC.Z_tot,f_arr)
impedancePlot(plt.figure(), f_arr, Z_Z1_MC, note='Z_Z1_MC', name='Z1_MC.png')


 makes some cross check on the bahavior of the Z1_MC contributions

In [ ]:
Z1_MC_det100pf = Q.Z1_MC('Z1_MC_det100pf',Cdet=100.0e-12)
Z1_MC_det10pf = Q.Z1_MC('Z1_MC_det10pf',Cdet=10.0e-12)
Z1_MC_Rbias10Meg = Q.Z1_MC('Z1_MC_Rbias10Meg',Rbias=10.0e6)
Z_Z1_MC_det100pf = to_np_array(Z1_MC_det100pf.Z_tot,f_arr)
Z_Z1_MC_det10pf = to_np_array(Z1_MC_det10pf.Z_tot,f_arr)
Z_Z1_MC_Rbias10Meg = to_np_array(Z1_MC_Rbias10Meg.Z_tot,f_arr)
fig = plt.figure()
impedancePlot(fig, f_arr, Z_Z1_MC, legend='Z_Z1_MC')
impedancePlot(fig, f_arr, Z_Z1_MC_det100pf, legend='Z_Z1_MC_det100pf')
impedancePlot(fig, f_arr, Z_Z1_MC_det10pf, legend='Z_Z1_MC_det10pf')
impedancePlot(fig, f_arr, Z_Z1_MC_Rbias10Meg, legend='Z_Z1_MC_Rbias10Meg', name='Z1_MC_cmp.png')



The MC stage contribution is connected in series with 4K stage through Cc

In [ ]:
Z_Z1_MC_g = Q.series(Z_Z1_g, Z_Z1_MC)
#Z_Z1_MC_g = Z_Z1_MC #Q.series(Z_Z1_g, Z_Z1_MC)
fig = plt.figure()
impedancePlot(fig, f_arr, Z_Z1_MC_g, note='Z_Z1_MC_g', name='Z1_MC_g.png')

fig = plt.figure()
impedancePlot(fig, f_arr, Z_Z1_MC, legend='Z_Z1_MC')
impedancePlot(fig, f_arr, Z_Z1_g, legend='Z_Z1_g')
impedancePlot(fig, f_arr, Z_Z1_MC_g, legend='Z_Z1_MC_g', name='Z1_MC_cmp.png')



The feedback signal are split between the HEMT input impedance and gate coupling capactor

In [ ]:
Z_input = Q.parallel(Z_HEMT, Z_Z1_MC_g)
fig = plt.figure()
impedancePlot(fig, f_arr, Z_input, note='Z_input', name='Z_input.png')

Compare w/ and w/o MC stage

In [ ]:
fig = plt.figure()
impedancePlot(fig, f_arr, Z_input, legend='Z_input')
impedancePlot(fig, f_arr, Z_input_4K, legend='Z_input_4K')
impedancePlot(fig, f_arr, Z_Z1_MC_g, legend='Z1_MC',name='Z_input_cmp.png')

Feedback fraction and input damping

In [ ]:
# feedback fraction
H_fb_det = Z_input/(Z_input + Z_Z2)
fig = plt.figure()
impedancePlot(fig, f_arr, H_fb_det, legend=r'$H_{fb,det}$')

# input damping
H_in_det = -1*Z_Z2/(Z_input + Z_Z2)

impedancePlot(fig, f_arr, H_in_det, 'H_in_fb_det.png', note='FB and damping w/ det', legend=r'$H_{in,det}$')


### total closed loop voltage gain

In [ ]:
Atotal_closed_det = H_in_det*Atotal_open/(1+Atotal_open*H_fb_det)

fig = plt.figure()
impedancePlot(fig, f_arr, Atotal_closed_det, 'Atotal_closed_det.png', ylabel='Atotal closed det', note='Atotal_closed_det')

# compare w/ and w/o MC stage
fig = plt.figure()
impedancePlot(fig, f_arr, Atotal_closed_det, legend='Total closed w/ det')
impedancePlot(fig, f_arr, Atotal_closed, legend='Total closed',name='Atotal_closed_det_cmp.png',ylabel='Gain')


## Noise calculation


In [ ]:


# noise from feedback Circuit
en_Z2 = np.sqrt(4*T_4K*Q.kB*Z_Z2.real)

fig = plt.figure()
noisePlot(fig, f_arr, en_Z2, name='en_Z2', ylabel='V/sqHz')

en_Z2_input = en_Z2/Atotal_closed

fig = plt.figure()
noisePlot(fig, f_arr, en_Z2_input, name='en_Z2_gain', ylabel='V/sqHz')

# noise from HEMT
en_HEMT = hemt.voltageNoise(f_arr, fc=1.2e3, vflat=0.24e-9)

fig = plt.figure()
noisePlot(fig, f_arr, en_HEMT,  name='en_HEMT', ylabel='V/sqHz')

# noise from Z3
en_Z3 = np.sqrt(4*T_300K*Q.kB*Z_Z3.real)

fig = plt.figure()
noisePlot(fig, f_arr, en_Z3, ylabel='V/sqHz', legend=r'$e_{n}$(Z$_{3})$')

# check this!
en_Z3_input = en_Z3/Aopen_HEMT

fig = plt.figure()
noisePlot(fig, f_arr, en_Z3_input,  name='en_Z3_input', ylabel=r'$V/\sqrt{Hz}$', legend=r'$e_{n}(Z_{3})$ (input)')



# noise from BJT
in_BJT = np.ones(len(f_arr))*bjt.currentNoise(drainI)

fig = plt.figure()
noisePlot(fig, f_arr, in_BJT, name='in_BJT', ylabel=r'A/\sqrt{Hz}', note='BJT shot noise')

# voltage noise as referred to the input of HEMT (scale by gm)
en_BJT = in_BJT/hemt.gm

fig = plt.figure()
noisePlot(fig, f_arr, en_BJT, name='en_BJT', ylabel=r'V/\sqrt{Hz}', note=r'BJT shot noise (input, gm={0:.1f}mS)'.format(hemt.gm*1e3))
#impedancePlot(fig, f_arr, Aopen_HEMT, 'Aopen_HEMT.png', ylabel='Aopen HEMT', note='Aopen HEMT gm={0:.1f}mS'.format(hemt.gm*1e3))


